In [1]:
import easyocr
 
image_path = 'data/sample.jpg'
# 言語選択（デフォルトでGPU利用、CPUのみの場合はgpu=False）
reader = easyocr.Reader(['ja','en'], gpu=True)
result = reader.readtext(image_path)#, detail=0)

In [2]:
print(result[:5])
# print('\n'.join([r[1] for r in result]))

[([[385, 109], [475, 109], [475, 141], [385, 141]], 'Coaking', 0.850903059635093), ([[1215, 104], [1349, 104], [1349, 147], [1215, 147]], '冷蔵保存', 0.9996510148048401), ([[1364, 110], [1462, 110], [1462, 147], [1364, 147]], '2-3日', 0.9538767337799072), ([[1492, 105], [1626, 105], [1626, 154], [1492, 154]], '冷凍保存', 0.9939463138580322), ([[1651, 114], [1731, 114], [1731, 152], [1651, 152]], '功カ月', 0.09941499680280685)]


In [3]:
# !pip install scikit-learn

In [4]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import numpy as np

def cluster_ocr_results(ocr_results, eps=0.4, min_samples=2):
    # Bounding boxの中心座標とテキスト情報を計算してデータを作成
    centers = []
    texts = []
    confidences = []
    for i, result in enumerate(ocr_results):
        box = result[0]
        center = np.mean(box, axis=0)
        centers.append(center)
        texts.append(result[1])
        confidences.append(result[2])
    X = StandardScaler().fit_transform(centers)

    # DBSCANによるクラスタリング
    dbscan = DBSCAN(eps=eps, min_samples=min_samples).fit(X)

    # クラスタリング結果と各クラスタに含まれるテキスト情報の出力
    labels = dbscan.labels_
    print(set(labels))
    clusters = []
    for label, result in zip(labels, ocr_results):
        result_dict = {'box': result[0], 'text': result[1], 'confidence': result[2], 'label': label}
        clusters.append(result_dict)
    return clusters

clustered_result = cluster_ocr_results(result)

{0, 1, 2, 3, 4, -1}


In [5]:
clusters = {}
for r in clustered_result:
    if r['label'] not in clusters.keys():
        clusters[r['label']] = []
    clusters[r['label']].append(r)

print(clusters.keys())

recipe_text = ''
for label,cluster in clusters.items():
    for c in cluster:
        recipe_text += c['text']+'\n'
    recipe_text += '\n'

print(recipe_text)

dict_keys([0, 1, -1, 2, 3, 4])
Coaking
Tume
調理時間
1Omin

冷蔵保存
2-3日
冷凍保存
功カ月
小松菜と豚バラの
塩しょうが蒸し
小松菜を1束たっぷり使って 豚バラと重ねて蒸し焼きに。
シンプルな味つけですが 最後に加えるしょうゆで香りと
コクが増し グンとおいしく!
材料(2人分)
豚バラ薄切り肉
酒大さじ3
(しゃぶしゃぶ用)
塩ひとつまみ
小松菜…1束 (2009)
しょうが(チューブ)
ごま油‥大さじ1/2
…1~Zcm
白州りごま (好みで)適量
かつお節 (あれば)1袋 (2.59)
しょうゆ ‥小さじ1
…2009

お弁当向け
作り方
を加えて
047

10分
~
じいた
める。
仕上
1o
器に盛
んはきの
スープの

調理POINT
代用食材
豚肉が長い場合は食べや
すい大きさに切ってください。
豚バラ薄切り肉は豚ロー
小松菜の苦味が苦手な方
ス肉や豚こま切れ肉 鶏も
は
@を加えるタイミングで白
も肉でもOKo
小松菜はち
すりごま大さじ1~2を加える
んげん菜 キャベツ 白菜
と
より食べやすくなります。
プロッコリーで代用できます。
温め直し
|ラップをかけて温めモードで電子レンジ加熱。

@小松菜は3~4C長さのざく切りにする。
フライパンにごま油をひいて中火で熱し 豚肉を州める。
の豚肉の色が8割がた変わったら小松菜を加え
フタをし 5分ほど蒸し焼きにする。
@小松菜がしんなりしたら回を加え サッと州める。 器に盛
り
好みで白州りごまをふる。




In [6]:
openai_key = open('../open_ai.key','r').read()
import openai
openai.api_key = openai_key
def completion(new_message_text:str, settings_text:str = '', past_messages:list = []):
    """
    new_message_text: str ユーザ入力
    settings_text: str システム設定（ChatGPTのふるまいを指定する文言）
    past_messages: list ユーザとシステムのやりとりの履歴
    """
    if len(past_messages) == 0 and len(settings_text) != 0:
        system = {"role": "system", "content": settings_text}
        past_messages.append(system)
    new_message = {"role": "user", "content": new_message_text}
    past_messages.append(new_message)

    result = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=past_messages
    )
    response_message = {"role": "assistant", "content": result.choices[0].message.content}
    past_messages.append(response_message)
    response_message_text = result.choices[0].message.content
    return response_message_text, past_messages

In [7]:
import json
recipe_format = json.dumps({
    "title": "",
    "ingredients": [
        {"name": "", "amount": ""},
    ],
    "appliances": [""],
    "steps": [""],
    "cooking_time": "",
    "junre": [""],
    "nutrition": {
        "calorie": 0.0,
        "protein": 0.0,
        "fat": 0.0,
        "carbohydrate": 0.0,
        "sodium": 0.0,
        "vitamin_a": 0.0,
        "vitamin_b1": 0.0,
        "vitamin_b2": 0.0,
        "vitamin_c": 0.0,
        "vitamin_e": 0.0,
        "vitamin_k": 0.0,
        "vitamin_pp": 0.0,
        "vitamin_b6": 0.0,
        "vitamin_b12": 0.0,
        "vitamin_d": 0.0,
    },
})


In [8]:
system_settings = """レシピテキストをJson形式のレシピデータを生成してください。
では、会話を開始します。
"""

In [9]:
new_message, messages = completion(
    f"以下のレシピを{recipe_format}に従って整形してください。\n\n{recipe_text}",
    system_settings, [])
print(new_message)

{
    "title": "小松菜と豚バラの塩しょうが蒸し",
    "ingredients": [
        {"name": "豚バラ薄切り肉", "amount": "しゃぶしゃぶ用"},
        {"name": "酒", "amount": "大さじ3"},
        {"name": "塩", "amount": "ひとつまみ"},
        {"name": "小松菜", "amount": "1束"},
        {"name": "しょうが(チューブ)", "amount": "1~2cm"},
        {"name": "ごま油", "amount": "大さじ1/2"},
        {"name": "白ごま", "amount": "適量（好みで）"},
        {"name": "かつお節", "amount": "1袋（あれば）"},
        {"name": "しょうゆ", "amount": "小さじ1"}
    ],
    "appliances": [
        "フライパン"
    ],
    "steps": [
        "小松菜は3~4cm長さのざく切りにする。",
        "フライパンにごま油をひいて中火で熱し、豚肉を炒める。",
        "豚肉の色が8割がた変わったら小松菜を加え、フタをして5分ほど蒸し焼きにする。",
        "小松菜がしんなりしたら、しょうがとしょうゆを加え、サッと炒める。",
        "器に盛り、好みで白ごまをふる。"
    ],
    "cooking_time": "10分",
    "junre": [
        "蒸し物",
        "副菜"
    ],
    "nutrition": {
        "calorie": 0.0,
        "protein": 0.0,
        "fat": 0.0,
        "carbohydrate": 0.0,
        "sodium": 0.0,
        "vitamin_a": 0.0,
        "vitamin_b1": 0.0,
        

In [10]:
import re
pattern = r'\{.*\}'

# 正規表現を使用してJSON部分のみを抽出する
match = re.search(pattern, new_message, re.DOTALL)
if match:
    json_string = match.group()
    json_obj = json.dumps(json.loads(json_string),indent=4,ensure_ascii=False)
    print(json_obj)

{
    "title": "小松菜と豚バラの塩しょうが蒸し",
    "ingredients": [
        {
            "name": "豚バラ薄切り肉",
            "amount": "しゃぶしゃぶ用"
        },
        {
            "name": "酒",
            "amount": "大さじ3"
        },
        {
            "name": "塩",
            "amount": "ひとつまみ"
        },
        {
            "name": "小松菜",
            "amount": "1束"
        },
        {
            "name": "しょうが(チューブ)",
            "amount": "1~2cm"
        },
        {
            "name": "ごま油",
            "amount": "大さじ1/2"
        },
        {
            "name": "白ごま",
            "amount": "適量（好みで）"
        },
        {
            "name": "かつお節",
            "amount": "1袋（あれば）"
        },
        {
            "name": "しょうゆ",
            "amount": "小さじ1"
        }
    ],
    "appliances": [
        "フライパン"
    ],
    "steps": [
        "小松菜は3~4cm長さのざく切りにする。",
        "フライパンにごま油をひいて中火で熱し、豚肉を炒める。",
        "豚肉の色が8割がた変わったら小松菜を加え、フタをして5分ほど蒸し焼きにする。",
        "小松菜がしんなりしたら、しょうがとしょうゆを加え、サッと炒める。",
    

In [11]:
new_message, messages = completion(
    f"以下のレシピデータについて、栄養素などを推定して完成させてください。フォーマットは{recipe_format}に従って整形してください。ingredientsには、'g'をキーとして、分量をグラムに変換して追加してください。\n{json_obj}",
    system_settings, messages)
print(new_message)

{
    "title": "小松菜と豚バラの塩しょうが蒸し",
    "ingredients": [
        {
            "name": "豚バラ薄切り肉",
            "amount": "しゃぶしゃぶ用",
            "g": 80
        },
        {
            "name": "酒",
            "amount": "大さじ3",
            "g": 45
        },
        {
            "name": "塩",
            "amount": "ひとつまみ",
            "g": 1.5
        },
        {
            "name": "小松菜",
            "amount": "1束",
            "g": 200
        },
        {
            "name": "しょうが(チューブ)",
            "amount": "1~2cm",
            "g": 5
        },
        {
            "name": "ごま油",
            "amount": "大さじ1/2",
            "g": 7.5
        },
        {
            "name": "白ごま",
            "amount": "適量（好みで）",
            "g": 0
        },
        {
            "name": "かつお節",
            "amount": "1袋（あれば）",
            "g": 2.59
        },
        {
            "name": "しょうゆ",
            "amount": "小さじ1",
            "g": 5
        }
    ],
    "appliances": [
        "フライパン"

In [12]:
import re
pattern = r'\{.*\}'

# 正規表現を使用してJSON部分のみを抽出する
match = re.search(pattern, new_message, re.DOTALL)
if match:
    json_string = match.group()
    json_obj = json.dumps(json.loads(json_string),indent=4,ensure_ascii=False)
    print(json_obj)

{
    "title": "小松菜と豚バラの塩しょうが蒸し",
    "ingredients": [
        {
            "name": "豚バラ薄切り肉",
            "amount": "しゃぶしゃぶ用",
            "g": 80
        },
        {
            "name": "酒",
            "amount": "大さじ3",
            "g": 45
        },
        {
            "name": "塩",
            "amount": "ひとつまみ",
            "g": 1.5
        },
        {
            "name": "小松菜",
            "amount": "1束",
            "g": 200
        },
        {
            "name": "しょうが(チューブ)",
            "amount": "1~2cm",
            "g": 5
        },
        {
            "name": "ごま油",
            "amount": "大さじ1/2",
            "g": 7.5
        },
        {
            "name": "白ごま",
            "amount": "適量（好みで）",
            "g": 0
        },
        {
            "name": "かつお節",
            "amount": "1袋（あれば）",
            "g": 2.59
        },
        {
            "name": "しょうゆ",
            "amount": "小さじ1",
            "g": 5
        }
    ],
    "appliances": [
        "フライパン"